In [1]:
%pylab inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
set_printoptions(linewidth=300, threshold=200000, suppress=True, precision = 8)


%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/tmp/ipykernel_1483657/1024220006.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
figsize(16,9)

def plant_biomass_logistic(dt, NB, p):
    NB[...] = clip(NB, 0, 1e9)
    rn, rb, B0, Kb, eta, Si, H, L = p
    N, B = NB
    #print(N, B, Si, rn, B/N, B0, B/N - B0, rn * (B / N - B0) * N)
    dN = rn * (B / N - B0) * N + Si
    dB = rb * (N/(eta + N)) * B * (1 - B / Kb) - H - L
    return array([dN, dB]) * dt

def plant_biomass_logistic_lv(dt, NB, p):
    NB[...] = clip(NB, 0, 1e9)
    rn, rb, B0, Kb, eta, Si, H, L, rnw, rbw, B0w, Kbw, etaw, c, cw = p
    N, B, Nw, Bw = NB
    #print(N, B, Si, rn, B/N, B0, B/N - B0, rn * (B / N - B0) * N)
    dN = rn * (B / N - B0) * N + Si
    dB = rb * (N/(eta + N)) * B * (1 - B / (Kb - c * Bw)) - H - L
    dNw = rnw * (Bw / Nw - B0w) * Nw
    dBw = rbw * (Nw/(eta + Nw)) * Bw * (1 - Bw / (Kbw - cw * B))
    return array([dN, dB, dNw, dBw]) * dt

def rk4(f, dt, y, p):
    dx = dt
    k1 = f(dx, y, p)
    k2 = f(dx, y + k1 / 2, p)
    k3 = f(dx, y + k2 / 2, p)
    k4 = f(dx, y + k3, p)
    return y + (k1 + 2 * k2 + 2 * k3 + k4) / 6


def simula(funcion, nt = 120, dt = 1e-3, N = 1, B = 1, rn = 0.1, rb = 0.1, B0 = 1.0, Kb = 10, eta = 0, Si = 0, H = 0, L = 0):
    #punto critico en Kb/2
    #por defecto el maximo de extraccion (dbmax) es 0.1 * 10 / 2 = 0.5
    #pero con el termino de MM, eta > 0,  es 0.1 * 10 / 2 * (1 / (eta + 1))
    #con eta = 1, dbmax es igual a 0.25
    t = arange(0, nt, dt)
    N = array([N, B])
    M = zeros((t.shape[0], N.shape[0] + 1))
    Kb = zeros_like(t) + Kb

    M[:,-1] = t
    for i in range(t.shape[0]):
        M[i,:-1] = N
        N = clip(rk4(funcion, dt, N, (rn, rb, B0, Kb[i], eta, Si, H, L)), 0, 1e99)
    return M

def simula_lv(funcion, nt = 120, dt = 1e-3, N = 10, B = 10, rn = 0.1, rb = 0.1, B0 = 1.0, Kb = 10, eta = 1, Si = 0, H = 0, L = 0,
              Nw = 10, Bw = 10, rnw = 0.05, rbw = 0.04, B0w = 1, Kbw = 10, etaw = 1, c = 1, cw = 1):
    t = arange(0, nt, dt)
    N = array([N, B, Nw, Bw])
    M = zeros((t.shape[0], N.shape[0] + 1))
    Kb = zeros_like(t) + Kb
    
    M[:,-1] = t
    for i in range(t.shape[0]):
        M[i,:] = N
        N = clip(rk4(funcion, dt, N, (rn, rb, B0, Kb[i], eta, Si, H, L, rnw, rbw, B0w, Kbw, etaw, c, cw)), 0, 1e99)
    return M